# Stacks & Queues — Practice Notebook

This notebook gives you hands‑on practice implementing **Stacks** (LIFO) and **Queues** (FIFO) in multiple ways. Note there are severaal difference in the way things are done in this notebook compared to the standarrd python syntax we have seen in previous lectures. 

- the dataclass decorator automatically generates boilerplate methods (like __init__, __repr__, and __eq__) for classes that mainly store data, making them cleaner and easier to maintain. 

- The syntax uses typin module to define data types. the typing module lets us annotate variables and function signatures with types (e.g., int, str, List[str]), which doesn’t enforce types at runtime but helps with readability, tooling support (like IDE autocompletion), and static analysis—making our programs safer and easier to reason about.

In [2]:

from __future__ import annotations
from dataclasses import dataclass
from typing import Any, Optional, Iterable, List

## Part 1 — Stacks refresher (LIFO)
A **stack** lets you add and remove from **one end**: the last item pushed is the first item popped (**LIFO**).
Core operations in stacks are: 
- `push(x)`
- `pop() -> x`
- `peek() -> x`.

###  Stack via Python list (naïve but handy)

In Python, `list.append(x)` behaves like `push`, and `list.pop()` removes the last element.  
Implement a thin `ListStack` wrapper and try the provided tests. In Python, a wrapper is a function, class, or object that provides an interface around another function or object, usually to add functionality without changing the original code. Here the ListStack class is a wrapper we build on top of the python List object.

In [3]:
class ListStack:

    def __init__(self):
        self._items: List[Any] = []
        self.size = 0

    def push(self, data: Any) -> None:
        self._items.append(data)
        self.size += 1

    def pop(self) -> Optional[Any]:
        if not self._items:
            return None
        self.size -= 1
        return self._items.pop()

    def peek(self) -> Optional[Any]:
        return self._items[-1] if self._items else None

    def __len__(self):
        return self.size
    
    @classmethod
    def description(cls) -> str:
        return (
            f"{cls.__name__}"
        )

## Quick checks

When we write code, it is a good idea to write a few assertions to verify that our method behaves correctly. Lets see how we can design an quick check to assert our stack class:
- After pushing [1, 2, 3], the length should be 3 and peek() should return the last item 3.
- Popping three times should return 3 → 2 → 1 in reverse order (LIFO behavior).
- A fourth pop() on an empty stack should return None, and peek() should also return None, with the length back to 0.

If all assertions pass, the message "basic tests: OK" is printed, confirming the stack implementation works as expected.

In [4]:
def quickcheckstack(stackclass):
    s = stackclass
    for x in [1,2,3]: s.push(x)
    assert len(s) == 3 and s.peek() == 3
    assert s.pop() == 3 and s.pop() == 2 and s.pop() == 1
    assert s.pop() is None and s.peek() is None and len(s) == 0
    print(f"{stackclass.description()} basic tests: OK")


quickcheckstack(ListStack())

ListStack basic tests: OK


### Stack via linked nodes
While the python List is a easy way to mimic behaviour of stacks, the problem with memoy with lists make it difficult to work with. Therefore to avoid memory hotspots, creating stacks with nodes (as we did for linked lists) is a better way.
the code block below, uses the @dataclass decorator to define the node class. This decorator automatically generates boilerplate methods (like __init__) for classes that mainly store data, making them cleaner and easier to maintain. 


In [5]:
@dataclass
class Node:
    data: Any
    next: Optional["Node"] = None

Next we create the LinkedStack class that we wiil use for our stack object. In the stack object we will maintain a pointer to the **top** node and define the following methods:
- `push` creates a new node whose `next` points to the current top; 
- `pop` returns the top and moves the pointer down.
- `peek` method that shows the data present in the top node
- `len` method that returns the size of our stack


In Python, defining the len method using __len__ allows us to use the built-in len(obj) function on our custom data structures. Instead of making Python count elements each time (which could be slow for linked structures), we maintain a running counter self.size that updates on every push and pop. Returning this counter in __len__ makes length checks efficient while keeping the stack interface consistent with Python’s built-in collections.

In [6]:
class LinkedStack:
    def __init__(self):
        self.top: Optional[Node] = None
        self.size: int = 0

    def push(self, data: Any) -> None:
        new_node = Node(data, self.top)
        self.top = new_node
        self.size += 1

    def pop(self) -> Optional[Any]:
        if self.top is None:
            return None
        data = self.top.data
        self.top = self.top.next
        self.size -= 1
        return data

    def peek(self) -> Optional[Any]:
        return self.top.data if self.top else None

    def __len__(self): 
        return self.size
    
    @classmethod
    def description(cls) -> str:
        return (
            f"{cls.__name__}"
        )



In [7]:
quickcheckstack(LinkedStack())

LinkedStack basic tests: OK


## Queues refresher (FIFO)
A **queue** serves items in **first-in, first-out** order. Core oprations we will look at are: 
- `enqueue(x)` (add at back), 
- `dequeue() -> x` (remove from front).

We’ll build queues in three different ways: 
- with a wrapper around python lists
- with the node data class
- using two stacks

### Wrapper on top of a Python list

We'll use the insert method `insert(0, x)` of the list insted of the append method to ensure new elements are added to front of list while the  `pop()` method removes from the end. 

In [8]:
class ListQueue:
    def __init__(self):
        self.items: List[Any] = []
        self.size = 0

    def enqueue(self, data: Any) -> None:
        self.items.insert(0, data)   # enqueue to the "back"
        self.size += 1

    def dequeue(self) -> Optional[Any]:
        if not self.items:
            return None
        self.size -= 1
        return self.items.pop()      # remove from the "front"

    def __len__(self):
        return self.size
    
    @classmethod
    def description(cls) -> str:
        return (
            f"{cls.__name__}"
        )



In [9]:
def quickcheckqueue(queueobj):
    q = queueobj
    for x in [10,20,30]: q.enqueue(x)
    assert len(q) == 3 and q.dequeue() == 10 and q.dequeue() == 20 and q.dequeue() == 30
    assert q.dequeue() is None and len(q) == 0
    print(f"{queueobj.description()}basic tests: OK")

In [10]:
quickcheckqueue(ListQueue())

ListQueuebasic tests: OK


### Queue via nodes (head/tail pointers)

We maintain both **head** (back/rear) and **tail** (front) pointers.  
- `enqueue(x)`: append at **head**; connect pointers.  
- `dequeue()`: remove at **tail** and update pointers.

In [11]:
@dataclass
class DNode:
    data: Any
    next: Optional["DNode"] = None
    prev: Optional["DNode"] = None

class LinkedQueue:
    def __init__(self):
        self.head: Optional[DNode] = None  # most recently enqueued (back)
        self.tail: Optional[DNode] = None  # next to be dequeued (front)
        self.count: int = 0

    def enqueue(self, data: Any) -> None:
        new_node = DNode(data)
        if self.tail is None:
            # empty queue
            self.tail = self.head = new_node
        else:
            # attach at head (back)
            new_node.prev = self.head
            self.head.next = new_node
            self.head = new_node
        self.count += 1

    def dequeue(self) -> Optional[Any]:
        if self.count == 0:
            return None
        data = self.tail.data
        if self.count == 1:
            self.tail = self.head = None
        else:
            self.tail = self.tail.next
            self.tail.prev = None
        self.count -= 1
        return data

    def __len__(self):
        return self.count
    
    
    @classmethod
    def description(cls) -> str:
        return (
            f"{cls.__name__}"
        )




In [12]:

lq = LinkedQueue()
quickcheckqueue(lq)

LinkedQueuebasic tests: OK


### Queue via **two stacks**

Maintain two stacks:  
- `inbound_stack` for enqueues,  
- `outbound_stack` for dequeues (refill it by popping all items from inbound when needed).

Here, we mimic both the stacks using python Lists.


In [13]:
class StackQueue:
    def __init__(self):
        self.inbound_stack: List[Any] = []
        self.outbound_stack: List[Any] = []

    def enqueue(self, data: Any) -> None:
        self.inbound_stack.append(data)

    def dequeue(self) -> Optional[Any]:
        if not self.outbound_stack:
            while self.inbound_stack:
                self.outbound_stack.append(self.inbound_stack.pop())
        if not self.outbound_stack:
            return None
        return self.outbound_stack.pop()




In [14]:
# Checks
sq = StackQueue()
for x in [1,2,3,4]:
    sq.enqueue(x)
assert sq.dequeue() == 1 and sq.dequeue() == 2
sq.enqueue(5); sq.enqueue(6)
assert sq.dequeue() == 3 and sq.dequeue() == 4 and sq.dequeue() == 5 and sq.dequeue() == 6
assert sq.dequeue() is None
print("StackQueue tests: OK")

StackQueue tests: OK


## Part 3 — Complexity (Big‑O) quick notes

| Structure | Enqueue/Push | Dequeue/Pop | Peek |
|---|---|---|---|
| ListStack | O(1) | O(1) | O(1) |
| LinkedStack | O(1) | O(1) | O(1) |
| ListQueue (`insert(0, x)`) | **O(n)** | O(1) | — |
| LinkedQueue | O(1) | O(1) | — |
| Two‑Stack Queue | Amortized O(1) | Amortized O(1) | — |

## Part 4 — Activities 


2. **Undo/Redo Simulator** (Two Stacks)  
   Implement an editor history with `do(action)`, `undo()`, `redo()` using two stacks.

3. **Reverse a Queue** (Stack + Queue)  
   Given a `LinkedQueue`, reverse it **in place** using a stack.

7. **Queue using Two Stacks — Instrumentation**  
   Modify `StackQueue` to show the lengh of the queue at any given time.

In [15]:
class UndoRedo:
    """
    Simple editor history using two stacks.
    - do(action): record a new action, clearing the redo stack
    - undo(): move the most recent action from undo -> redo, return it
    - redo(): move the most recent action from redo -> undo, return it
    """
    def __init__(self):
        self._undo = []   # actions already done (top = most recent)
        self._redo = []   # actions undone and available to redo

    def do(self, action: str) -> None:
        self._undo.append(action)
        self._redo.clear()  # once a new action is done, the redo history resets

    def undo(self):
        if not self._undo:
            return None
        a = self._undo.pop()
        self._redo.append(a)
        return a

    def redo(self):
        if not self._redo:
            return None
        a = self._redo.pop()
        self._undo.append(a)
        return a

    def undo_count(self) -> int:
        return len(self._undo)

    def redo_count(self) -> int:
        return len(self._redo)


In [16]:
h = UndoRedo()
h.do("type A"); h.do("type B"); h.do("delete B")
assert h.undo() == "delete B"
assert h.undo() == "type B"
assert h.redo() == "type B"
h.do("type C")            # new action clears redo branch
assert h.redo() is None
assert h.undo() == "type C"

## Why this works 

Think of two boxes:

- One box called Undo – this keeps track of everything you’ve done.

- Another box called Redo – this keeps track of things you’ve undone (so you can bring them back later).

Here’s what happens step-by-step:

- When you do something new (like typing or deleting), we put it into the Undo box.
If you do something new, the Redo box gets emptied — because the “future” actions you could redo no longer make sense.

- When you press Undo, we take the most recent thing from the Undo box and move it to the Redo box.
That’s like saying “I’m taking this back, but I might want to redo it later.”

- When you press Redo, we take the top thing from the Redo box and put it back into the Undo box.
That means “I’ve decided to do that action again.”

So basically:

- “Undo” moves things from Undo → Redo

- “Redo” moves things from Redo → Undo

- “Do something new” clears the Redo box

In [17]:
def reverse_queue(q: "LinkedQueue") -> None:
    """
    Reverse the queue in-place (logical in-place: same queue object) using a stack.
    Time: O(n), Space: O(n)
    """
    stack = []
    # Dequeue all items into the stack (front -> top)
    while len(q) > 0:
        stack.append(q.dequeue())
    # Pop from stack back into queue (top -> back); order is reversed
    while stack:
        q.enqueue(stack.pop())

In [18]:
lq = LinkedQueue()
for x in [1,2,3,4,5]: lq.enqueue(x)
reverse_queue(lq)
out = [lq.dequeue(), lq.dequeue(), lq.dequeue(), lq.dequeue(), lq.dequeue()]
assert out == [5,4,3,2,1]

## Why this works

When we take items out of the queue (dequeue), we put them into a stack.

A stack always gives items back in the opposite order (Last In, First Out).

So, when we take the items out of the stack and put them back into the queue, they go in backwards.

That’s why the queue ends up reversed — the first thing that went in becomes the last thing that comes out!

In [19]:
class StackQueue:
    """
    Queue implemented with two stacks, instrumented with length.
    """
    def __init__(self):
        self.inbound_stack = []
        self.outbound_stack = []
        self._size = 0

    def enqueue(self, data):
        self.inbound_stack.append(data)
        self._size += 1

    def _refill(self):
        if not self.outbound_stack:
            while self.inbound_stack:
                self.outbound_stack.append(self.inbound_stack.pop())

    def dequeue(self):
        self._refill()
        if not self.outbound_stack:
            return None
        self._size -= 1
        return self.outbound_stack.pop()

    def __len__(self):
        return self._size

In [20]:
sq = StackQueue()
assert len(sq) == 0
for x in [1,2,3,4]: sq.enqueue(x)
assert len(sq) == 4
assert sq.dequeue() == 1 and len(sq) == 3
sq.enqueue(5); sq.enqueue(6)
assert len(sq) == 5
_ = [sq.dequeue() for _ in range(5)]
assert len(sq) == 0 and sq.dequeue() is None

## Why this works:


We already know a two-stack queue is amortized O(1) for enqueue/dequeue. To “show the length,” just maintain a counter: increment on each successful enqueue, decrement when dequeue actually returns an element. __len__ then returns this counter in O(1).